In [1]:
# Import libraries
import pandas as pd
import numpy as np
import os
path = r'/Users/jakubkondelka/Documents/CF_Instacart Basket Analysis'


In [4]:
op_merged = pd.read_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'orders_products_merged_48.pkl'))

In [8]:
# check w head 
op_merged.head()

,order_id,user_id,evaluations,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,...,aisle_id,department_id,prices,_merge,price_range_loc,busiest day,Busiest days,Busiest_period_of_day,max_order,loyalty_flag
0,2539329,1,prior,1,2,8,NaN,196,1,0,...,77,7,9.0,both,Mid-range product,Regularly busy,Regularly busy,Most orders,10,New customer
1,2398795,1,prior,2,3,7,15.0,196,1,1,...,77,7,9.0,both,Mid-range product,Regularly busy,Least busy,Fewest orders,10,New customer
2,473747,1,prior,3,3,12,21.0,196,1,1,...,77,7,9.0,both,Mid-range product,Regularly busy,Least busy,Most orders,10,New customer
3,2254736,1,prior,4,4,7,29.0,196,1,1,...,77,7,9.0,both,Mid-range product,Least busy,Least busy,Fewest orders,10,New customer
4,431534,1,prior,5,4,15,28.0,196,1,1,...,77,7,9.0,both,Mid-range product,Least busy,Least busy,Most orders,10,New customer


In [7]:
# check shape
op_merged.shape

(32404859, 21)

## 2) Find the aggregated mean of the “order_number” column grouped by “department_id”

In [9]:
# aggregate by MEAN of order_number and group by column department_id
op_merged.groupby('department_id').agg({'order_number': ['mean']})

,order_number
,mean
department_id,
1,15.457838
2,17.277920
3,17.170395
4,17.811403
5,15.215751
6,16.439806
7,17.225802
8,15.340650


The results of running this on full data frame vs a subset is that hear every department is grouped by and each has a mean of order numbers. The subset had a size of only milion rows, therefore not all departments were part of the result.
We can also see that maximum MEAN of subset was 18,5 and with full data set it is 22.9

## 5) Use the loyalty flag you created and check the basic statistics of the product prices for each loyalty category (Loyal Customer, Regular Customer, and New Customer). 

In [16]:
# I want to group by Loyalty category and find min,max,mean of product prices 
op_merged.groupby('loyalty_flag').agg({'prices': ['mean', 'min', 'max']})

prices              
                       mean  min      max
loyalty_flag                             
Loyal customer    10.386336  1.0  99999.0
New customer      13.294670  1.0  99999.0
Regular customer  12.495717  1.0  99999.0

we can see the loayl customers buy on average the cheapest products. Than it is regular customer and the new customer is spending the most

## 6) The team now wants to target different types of spenders in their marketing campaigns. This can be achieved by looking at the prices of the items people are buying. Create a spending flag for each user based on the average price across all their orders using the following criteria:
If the mean of the prices of products purchased by a user is lower than 10, then flag them as a “Low spender.”
If the mean of the prices of products purchased by a user is higher than or equal to 10, then flag them as a “High spender.”

In [22]:
# use transform to aggregate prices by Mean. Group by user_id and prices

op_merged['mean_prices'] = op_merged.groupby(['user_id'])['prices'].transform(np.mean)

In [28]:
op_merged.head()

,order_id,user_id,evaluations,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,...,prices,_merge,price_range_loc,busiest day,Busiest days,Busiest_period_of_day,max_order,loyalty_flag,max_prices,mean_prices
0,2539329,1,prior,1,2,8,NaN,196,1,0,...,9.0,both,Mid-range product,Regularly busy,Regularly busy,Most orders,10,New customer,6.367797,6.367797
1,2398795,1,prior,2,3,7,15.0,196,1,1,...,9.0,both,Mid-range product,Regularly busy,Least busy,Fewest orders,10,New customer,6.367797,6.367797
2,473747,1,prior,3,3,12,21.0,196,1,1,...,9.0,both,Mid-range product,Regularly busy,Least busy,Most orders,10,New customer,6.367797,6.367797
3,2254736,1,prior,4,4,7,29.0,196,1,1,...,9.0,both,Mid-range product,Least busy,Least busy,Fewest orders,10,New customer,6.367797,6.367797
4,431534,1,prior,5,4,15,28.0,196,1,1,...,9.0,both,Mid-range product,Least busy,Least busy,Most orders,10,New customer,6.367797,6.367797


In [31]:
# if condtions to flag based on High Spender - mean > 10. Low Spender - mean < 10

op_merged.loc[op_merged['mean_prices'] < 10, 'Spender_flag'] = 'Low Spender'
op_merged.loc[op_merged['mean_prices'] > 10, 'Spender_flag'] = 'High Spender'

In [34]:
#check head
op_merged.head(100)

,order_id,user_id,evaluations,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,...,_merge,price_range_loc,busiest day,Busiest days,Busiest_period_of_day,max_order,loyalty_flag,max_prices,mean_prices,Spender_flag
0,2539329,1,prior,1,2,8,NaN,196,1,0,...,both,Mid-range product,Regularly busy,Regularly busy,Most orders,10,New customer,6.367797,6.367797,Low Spender
1,2398795,1,prior,2,3,7,15.0,196,1,1,...,both,Mid-range product,Regularly busy,Least busy,Fewest orders,10,New customer,6.367797,6.367797,Low Spender
2,473747,1,prior,3,3,12,21.0,196,1,1,...,both,Mid-range product,Regularly busy,Least busy,Most orders,10,New customer,6.367797,6.367797,Low Spender
3,2254736,1,prior,4,4,7,29.0,196,1,1,...,both,Mid-range product,Least busy,Least busy,Fewest orders,10,New customer,6.367797,6.367797,Low Spender
4,431534,1,prior,5,4,15,28.0,196,1,1,...,both,Mid-range product,Least busy,Least busy,Most orders,10,New customer,6.367797,6.367797,Low Spender
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,3226575,360,prior,1,5,12,NaN,196,1,0,...,both,Mid-range product,Regularly busy,Regularly busy,Most orders,3,New customer,10.006250,10.006250,High Spender
96,1469869,377,prior,3,5,17,3.0,196,9,0,...,both,Mid-range product,Regularly busy,Regularly busy,Most orders,3,New customer,8.496552,8.496552,Low Spender
97,1927023,387,prior,2,4,10,22.0,196,3,0,...,both,Mid-range product,Least busy,Least busy,Most orders,8,New customer,7.396610,7.396610,Low Spender
98,858092,420,prior,4,1,19,30.0,196,2,0,...,both,Mid-range product,Regularly busy,Busiest day,Average orders,22,Regular customer,7.387805,7.387805,Low Spender


In [35]:
#check shape
op_merged.shape

(32404859, 24)

In [36]:
# delete max_prices columns - it was a mistake 
op_merged = op_merged.drop(columns = ['max_prices'])

In [38]:
# check head
op_merged.head()

,order_id,user_id,evaluations,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,...,prices,_merge,price_range_loc,busiest day,Busiest days,Busiest_period_of_day,max_order,loyalty_flag,mean_prices,Spender_flag
0,2539329,1,prior,1,2,8,NaN,196,1,0,...,9.0,both,Mid-range product,Regularly busy,Regularly busy,Most orders,10,New customer,6.367797,Low Spender
1,2398795,1,prior,2,3,7,15.0,196,1,1,...,9.0,both,Mid-range product,Regularly busy,Least busy,Fewest orders,10,New customer,6.367797,Low Spender
2,473747,1,prior,3,3,12,21.0,196,1,1,...,9.0,both,Mid-range product,Regularly busy,Least busy,Most orders,10,New customer,6.367797,Low Spender
3,2254736,1,prior,4,4,7,29.0,196,1,1,...,9.0,both,Mid-range product,Least busy,Least busy,Fewest orders,10,New customer,6.367797,Low Spender
4,431534,1,prior,5,4,15,28.0,196,1,1,...,9.0,both,Mid-range product,Least busy,Least busy,Most orders,10,New customer,6.367797,Low Spender


In [51]:
# value counts
op_merged['Spender_flag'].value_counts(dropna = False)

Low Spender     31770614
High Spender      633864
NaN                  381
Name: Spender_flag, dtype: int64

## 7) In order to send relevant notifications to users within the app (for instance, asking users if they want to buy the same item again), the Instacart team wants you to determine frequent versus non-frequent customers. Create an order frequency flag that marks the regularity of a user’s ordering behavior according to the median in the “days_since_prior_order” column. The criteria for the flag should be as follows:
If the median of “days_since_prior_order” is higher than 20, then the customer should be labeled a “Non-frequent customer.”
If the median is higher than 10 and lower than or equal to 20, then the customer should be labeled a “Regular customer.”
If the median is lower than or equal to 10, then the customer should be labeled a “Frequent customer.”

In [40]:
# Create an order frequency flag that marks the regularity of a user’s ordering behavior according
# to the median in the “days_since_prior_order” column.



In [41]:
op_merged['order_frequency'] = op_merged.groupby(['user_id'])['days_since_prior_order'].transform(np.median)

In [43]:
# head
op_merged.head(20)

,order_id,user_id,evaluations,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,...,_merge,price_range_loc,busiest day,Busiest days,Busiest_period_of_day,max_order,loyalty_flag,mean_prices,Spender_flag,order_frequency
0,2539329,1,prior,1,2,8,NaN,196,1,0,...,both,Mid-range product,Regularly busy,Regularly busy,Most orders,10,New customer,6.367797,Low Spender,20.5
1,2398795,1,prior,2,3,7,15.0,196,1,1,...,both,Mid-range product,Regularly busy,Least busy,Fewest orders,10,New customer,6.367797,Low Spender,20.5
2,473747,1,prior,3,3,12,21.0,196,1,1,...,both,Mid-range product,Regularly busy,Least busy,Most orders,10,New customer,6.367797,Low Spender,20.5
3,2254736,1,prior,4,4,7,29.0,196,1,1,...,both,Mid-range product,Least busy,Least busy,Fewest orders,10,New customer,6.367797,Low Spender,20.5
4,431534,1,prior,5,4,15,28.0,196,1,1,...,both,Mid-range product,Least busy,Least busy,Most orders,10,New customer,6.367797,Low Spender,20.5
5,3367565,1,prior,6,2,7,19.0,196,1,1,...,both,Mid-range product,Regularly busy,Regularly busy,Fewest orders,10,New customer,6.367797,Low Spender,20.5
6,550135,1,prior,7,1,9,20.0,196,1,1,...,both,Mid-range product,Regularly busy,Busiest day,Most orders,10,New customer,6.367797,Low Spender,20.5
7,3108588,1,prior,8,1,14,14.0,196,2,1,...,both,Mid-range product,Regularly busy,Busiest day,Most orders,10,New customer,6.367797,Low Spender,20.5
8,2295261,1,prior,9,1,16,0.0,196,4,1,...,both,Mid-range product,Regularly busy,Busiest day,Most orders,10,New customer,6.367797,Low Spender,20.5
9,2550362,1,prior,10,4,8,30.0,196,1,1,...,both,Mid-range product,Least busy,Least busy,Most orders,10,New customer,6.367797,Low Spender,20.5


In [46]:
# if condtions to flag based regularity of visits

op_merged.loc[op_merged['order_frequency'] > 20, 'frequency_flag'] = 'Non frequent customer'
op_merged.loc[(op_merged['order_frequency'] <= 20) & (op_merged['order_frequency'] > 10), 'frequency_flag'] = 'Regular customer'
op_merged.loc[op_merged['order_frequency'] <= 10, 'frequency_flag'] = 'Frequent customer'

In [47]:
# chekc head 
op_merged.head(100)

,order_id,user_id,evaluations,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,...,price_range_loc,busiest day,Busiest days,Busiest_period_of_day,max_order,loyalty_flag,mean_prices,Spender_flag,order_frequency,frequency_flag
0,2539329,1,prior,1,2,8,NaN,196,1,0,...,Mid-range product,Regularly busy,Regularly busy,Most orders,10,New customer,6.367797,Low Spender,20.5,Non frequent customer
1,2398795,1,prior,2,3,7,15.0,196,1,1,...,Mid-range product,Regularly busy,Least busy,Fewest orders,10,New customer,6.367797,Low Spender,20.5,Non frequent customer
2,473747,1,prior,3,3,12,21.0,196,1,1,...,Mid-range product,Regularly busy,Least busy,Most orders,10,New customer,6.367797,Low Spender,20.5,Non frequent customer
3,2254736,1,prior,4,4,7,29.0,196,1,1,...,Mid-range product,Least busy,Least busy,Fewest orders,10,New customer,6.367797,Low Spender,20.5,Non frequent customer
4,431534,1,prior,5,4,15,28.0,196,1,1,...,Mid-range product,Least busy,Least busy,Most orders,10,New customer,6.367797,Low Spender,20.5,Non frequent customer
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,3226575,360,prior,1,5,12,NaN,196,1,0,...,Mid-range product,Regularly busy,Regularly busy,Most orders,3,New customer,10.006250,High Spender,4.0,Frequent customer
96,1469869,377,prior,3,5,17,3.0,196,9,0,...,Mid-range product,Regularly busy,Regularly busy,Most orders,3,New customer,8.496552,Low Spender,16.5,Regular customer
97,1927023,387,prior,2,4,10,22.0,196,3,0,...,Mid-range product,Least busy,Least busy,Most orders,8,New customer,7.396610,Low Spender,8.0,Frequent customer
98,858092,420,prior,4,1,19,30.0,196,2,0,...,Mid-range product,Regularly busy,Busiest day,Average orders,22,Regular customer,7.387805,Low Spender,7.0,Frequent customer


In [48]:
# check shape
op_merged.shape

(32404859, 25)

In [49]:
# check value counts
op_merged['frequency_flag'].value_counts(dropna = False)

Frequent customer        21559853
Regular customer          7208564
Non frequent customer     3636437
NaN                             5
Name: frequency_flag, dtype: int64

In [50]:
# we can see that 5 rows have missing data

In [52]:
# Export data to pkl
op_merged.to_pickle(os.path.join(path, '02 Data','Prepared Data', 'orders_products_merged_48task.pkl'))
